In [ ]:
# En caso de no tenerlo
# % pip install mlflow

In [1]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import joblib

Considerando que el proyecto fue hecho con la imagen de docker `rodolfojaramillo/project-ml:initial` disponible en dockerhub, y que la versión de python que usa es la 3.10.14, en Ubuntu, se requieren ejecutar algunos comandos que preparen el ambiente necesarios para que MLFlow cree el ambiente virtual donde el modelo sera servido.

Dentro de una terminal en el contenedor se corren las siguientes instrucciones. (Correrlo en la celda presentara un error)

In [ ]:
# Modulos necesarios para crear el ambiente
! apt-get update && apt-get install -y build-essential wget zlib1g-dev libffi-dev libssl-dev libsqlite3-dev
! export PATH="$HOME/.pyenv/bin:$PATH"
! eval "$(pyenv init --path)"

# Variables de ambiente necesarias para que MLFLow encuentre el modelo que se solicita.
! export MLFLOW_TRACKING_URI="https://dagshub.com/empleo-inducido/project-ml.mlflow"
! export MODEL_NAME="GBoost"
! export MODEL_VERSION=1

# Construir la variable MODEL_URI
! export MODEL_URI="models:/${MODEL_NAME}/${MODEL_VERSION}"

# Ejecutar el comando mlflow models serve
! mlflow models serve -m "$MODEL_URI" -p 5000

In [2]:
data = pd.read_parquet('/home/project/data/db/processed_test_data.parquet').sample(5).drop('TIPO_PACIENTE', axis = 1)

In [3]:
import requests
import pandas as pd
import json

Para hacer el request tiene que preparar los datos, una de las formas es mandando archivo json que sontenda los datos de la siguiente forma

In [4]:
data_json = pd.DataFrame({"dataframe_split": data.to_dict(orient="split")}).to_json()

Para el puerto por donde se accede al modelo, se tiene que usar la dirección ../invocations, además de los siguientes headers.

In [5]:
url = 'http://localhost:5000/invocations'
headers = {'Content-Type': 'application/json'}

Dentro de la respuesta al request esta un json, que contiene en 'predictions' los valores para registro de los datos simulados.

In [6]:
response = requests.post(url, data=data_json, headers=headers)

# Imprimir la respuesta
print(response.json())

{'predictions': [2, 2, 1, 1, 2]}
